<a href="https://colab.research.google.com/github/demekeendalie/Fine-tuning-bert-model-for-amharic-news-classification/blob/main/Bag_of_word_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
#importing basic packages
import pandas as pd
import numpy as np
import tensorflow as tf

In [85]:
# loading the drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [86]:
# reading the dataset from my drive using the above drive
dataframe1 =pd.read_excel('/content/drive/My Drive/dataset1.xlsx')
dataframe2 =pd.read_excel('/content/drive/My Drive/dataset2.xlsx')

In [87]:
# merging the two dataframes
frame=[dataframe1,dataframe2]
dataframe=pd.concat(frame)

In [88]:
print(dataframe)

                                              articles   category
0     ግንቦት 13/2011  በተሽከርካሪ አደጋ ድንገተኛ ህክምና የተፈቀደው ሁ...   accident
1    ሚያዝያ  25/ 2011 የሰሜን ተራሮች ብሄራዊ ፓርክን የአደጋ ስጋቶች ለ...   accident
2    በደቡብ ወሎ ዞን ኩታበር ወረዳ ዛሬ በደረሰ የትራፊክ አደጋ በ11 ሰዎች ...   accident
3    የህዝብ ተወካዮች ምክር ቤት አባላት ዛሬ ጠዋት የተፈናቀሉ ወገኖችን ለመጎ...   accident
4    በባህር ዳር ከተማ ቀበሌ 05 በእሳት አደጋ ቤትና ንብረት የወደመባቸው ሰ...   accident
..                                                 ...        ...
695  ጎባ፤ መስከረም 16/2015 (ኢዜአ) መደ ወላቡ ዩኒቨርሲቲ ተጨማሪ 13 ...  education
696  ባህር ዳር ሰኔ 7/2010 በአማራ ክልል ከህብረተሰቡ ከደረሱት ጥቆማዎች ...  education
697  አዲስ አበባ ሰኔ 29/2010 የትምህርት ጥራት ጉድለት በአገሪቱ ላይ ዘር...  education
698  አዲስ አበባ፤ መስከረም 15/ 2016 (ኢዜአ)፦ በተባበሩት አረብ ኤምሬቶ...  education
699  አዲስ አበባ መጋቢት 4/2012(ኢዜአ) ግማሽ ቢሊዮን ብር ወጪ የተደረገበ...  education

[2181 rows x 2 columns]


In [89]:
# changing category to numeric
label=pd.Categorical(dataframe['category']).codes
label=pd.DataFrame(label)
dataframe['labels']=pd.DataFrame(label)
print(dataframe)

                                              articles   category  labels
0     ግንቦት 13/2011  በተሽከርካሪ አደጋ ድንገተኛ ህክምና የተፈቀደው ሁ...   accident       0
1    ሚያዝያ  25/ 2011 የሰሜን ተራሮች ብሄራዊ ፓርክን የአደጋ ስጋቶች ለ...   accident       0
2    በደቡብ ወሎ ዞን ኩታበር ወረዳ ዛሬ በደረሰ የትራፊክ አደጋ በ11 ሰዎች ...   accident       0
3    የህዝብ ተወካዮች ምክር ቤት አባላት ዛሬ ጠዋት የተፈናቀሉ ወገኖችን ለመጎ...   accident       0
4    በባህር ዳር ከተማ ቀበሌ 05 በእሳት አደጋ ቤትና ንብረት የወደመባቸው ሰ...   accident       0
..                                                 ...        ...     ...
695  ጎባ፤ መስከረም 16/2015 (ኢዜአ) መደ ወላቡ ዩኒቨርሲቲ ተጨማሪ 13 ...  education       3
696  ባህር ዳር ሰኔ 7/2010 በአማራ ክልል ከህብረተሰቡ ከደረሱት ጥቆማዎች ...  education       3
697  አዲስ አበባ ሰኔ 29/2010 የትምህርት ጥራት ጉድለት በአገሪቱ ላይ ዘር...  education       3
698  አዲስ አበባ፤ መስከረም 15/ 2016 (ኢዜአ)፦ በተባበሩት አረብ ኤምሬቶ...  education       3
699  አዲስ አበባ መጋቢት 4/2012(ኢዜአ) ግማሽ ቢሊዮን ብር ወጪ የተደረገበ...  education       3

[2181 rows x 3 columns]


In [90]:
# mixing each categories and thier labels
dataframe=dataframe.sample(frac=1)

In [91]:
dataframe['word_len'] = dataframe['articles'].str.split().str.len()
print(dataframe['word_len'].mean())

314.2563044475011


In [92]:
#normalizarion
import re
#method to normalize character level missmatch such as ጸሀይ and ፀሐይ
def normalization(input_token):
    rep1=re.sub('[ሃኅኃሐሓኻ]','ሀ',input_token)
    rep2=re.sub('[ሑኁዅ]','ሁ',rep1)
    rep3=re.sub('[ኂሒኺ]','ሂ',rep2)
    rep4=re.sub('[ኌሔዄ]','ሄ',rep3)
    rep5=re.sub('[ሕኅ]','ህ',rep4)
    rep6=re.sub('[ኆሖኾ]','ሆ',rep5)
    rep7=re.sub('[ሠ]','ሰ',rep6)
    rep8=re.sub('[ሡ]','ሱ',rep7)
    rep9=re.sub('[ሢ]','ሲ',rep8)
    rep10=re.sub('[ሣ]','ሳ',rep9)
    rep11=re.sub('[ሤ]','ሴ',rep10)
    rep12=re.sub('[ሥ]','ስ',rep11)
    rep13=re.sub('[ሦ]','ሶ',rep12)
    rep14=re.sub('[ዓኣዐ]','አ',rep13)
    rep15=re.sub('[ዑ]','ኡ',rep14)
    rep16=re.sub('[ዒ]','ኢ',rep15)
    rep17=re.sub('[ዔ]','ኤ',rep16)
    rep18=re.sub('[ዕ]','እ',rep17)
    rep19=re.sub('[ዖ]','ኦ',rep18)
    rep20=re.sub('[ጸ]','ፀ',rep19)
    rep21=re.sub('[ጹ]','ፁ',rep20)
    rep22=re.sub('[ጺ]','ፂ',rep21)
    rep23=re.sub('[ጻ]','ፃ',rep22)
    rep24=re.sub('[ጼ]','ፄ',rep23)
    rep25=re.sub('[ጽ]','ፅ',rep24)
    rep26=re.sub('[ጾ]','ፆ',rep25)
    rep27=re.sub('(ሉ[ዋአ])','ሏ',rep26)
    rep28=re.sub('(ሙ[ዋአ])','ሟ',rep27)
    rep29=re.sub('(ቱ[ዋአ])','ቷ',rep28)
    rep30=re.sub('(ሩ[ዋአ])','ሯ',rep29)
    rep31=re.sub('(ሱ[ዋአ])','ሷ',rep30)
    rep32=re.sub('(ሹ[ዋአ])','ሿ',rep31)
    rep33=re.sub('(ቁ[ዋአ])','ቋ',rep32)
    rep34=re.sub('(ቡ[ዋአ])','ቧ',rep33)
    rep35=re.sub('(ቹ[ዋአ])','ቿ',rep34)
    rep36=re.sub('(ሁ[ዋአ])','ኋ',rep35)
    rep37=re.sub('(ኑ[ዋአ])','ኗ',rep36)
    rep38=re.sub('(ኙ[ዋአ])','ኟ',rep37)
    rep39=re.sub('(ኩ[ዋአ])','ኳ',rep38)
    rep40=re.sub('(ዙ[ዋአ])','ዟ',rep39)
    rep41=re.sub('(ጉ[ዋአ])','ጓ',rep40)
    rep42=re.sub('(ደ[ዋአ])','ዷ',rep41)
    rep43=re.sub('(ጡ[ዋአ])','ጧ',rep42)
    rep44=re.sub('(ጩ[ዋአ])','ጯ',rep43)
    rep45=re.sub('(ጹ[ዋአ])','ጿ',rep44)
    rep46=re.sub('(ፉ[ዋአ])','ፏ',rep45)
    rep47=re.sub('[ቊ]','ቁ',rep46) #ቁ can be written as ቊ
    rep48=re.sub('[ኵ]','ኩ',rep47) #ኩ can be also written as ኵ
    return rep48

In [93]:
# apply normalization and special characters removal
dataframe['articles']=dataframe['articles'].apply(lambda x: normalization(x))

In [94]:
# special symbol removal
dataframe['articles']=dataframe['articles'].str.replace('[^\w\s]','',regex=True)

In [95]:
# removing digits from the dataset
dataframe['articles']=dataframe['articles'].str.replace('\d+','',regex=True)

In [96]:
dataframe=dataframe[['articles','labels']]

In [97]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test,y_train,y_test = train_test_split(dataframe['articles'],dataframe['labels'], test_size=0.20, random_state=42)


In [100]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
# Create a Bag-of-Words   model
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# create mlp classifier
clf = MLPClassifier(hidden_layer_sizes=(100,),activation='relu', solver='adam', max_iter=1000)
clf.fit(X_train_vec, y_train)

# Predict on the test set
y_pred = clf.predict(X_test_vec)
# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, average='macro')) # Changed line: Added average='macro'
print("Recall:", recall_score(y_test, y_pred,average='macro')) # Changed line: Added average='macro'
print("F1-score:", f1_score(y_test, y_pred,average='macro')) # Changed line: Added average='macro'
print(classification_report(y_test, y_pred))

Accuracy: 0.816933638443936
Precision: 0.8239332973695637
Recall: 0.8261079101850827
F1-score: 0.8229460075637317
              precision    recall  f1-score   support

           0       0.77      0.83      0.80        70
           1       0.72      0.75      0.73        83
           2       0.79      0.66      0.72        50
           3       0.81      0.94      0.87        50
           4       0.88      0.79      0.83        86
           5       0.87      0.91      0.89        44
           6       0.92      0.91      0.92        54

    accuracy                           0.82       437
   macro avg       0.82      0.83      0.82       437
weighted avg       0.82      0.82      0.82       437

